In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!nvidia-smi

Mon Nov 13 23:13:42 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Nov 13 23:14:55 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
! pip install datasets transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 87.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.1/311.1 kB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 114.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 89.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 38.3 MB/s eta 0:00:00


In [ ]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 4.9 MB/s eta 0:00:00


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from transformers.utils import send_example_telemetry

send_example_telemetry("question_answering_notebook", framework="pytorch")

In [ ]:
squad_v2 = True 
model_checkpoint = "deepset/bert-base-cased-squad2"
batch_size = 16

In [ ]:
from datasets import load_dataset, load_metric

In [ ]:
#LOAD DATASET
import pickle
with open("/content/drive/MyDrive/Colab Notebooks/dataset/medical_dataset.pkl", "rb") as archivo: #"/content/sample_data/medical_dataset.pkl"
    datasets = pickle.load(archivo)

In [ ]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(datasets["train"])

,context,question,answers
0,"The patient is a 76-year-old female with a history of mitral regurgitation, congestive heart failure, recurrent UTIs, and uterine prolapse who presented with chills and hypotension and was admitted to the Medical ICU for treatment of septic shock. Mean arterial pressures were kept above 65 with Levophed and antibiotics were changed to penicillin 3 million units IV q.4h. and gentamicin 50 mg IV q.8h. An ATEE on 10/19 showed severe mitral regurgitation with posterior leaflet calcifications and linear density concerning for endocarditis, for which a PICC line was placed on 1/19 for a six-week course of penicillin 3 million units IV q.4h. and two-week course of gentamicin 50 mg IV q.8h. until 2/25. The patient was initially treated with Levophed for her hypotension until 11/0, and was placed on Levofloxacin and Vancomycin to treat Gram-positive cocci bacteremia and UTI. She was maintained on telemetry and was found to be a normal sinus rhythm with ectopy, including short once of nonsustained ventricular tachycardia. She was started on Lopressor 12.5 mg t.i.d. on 3/18, and this was increased to 25 mg b.i.d. at discharge, with her heart rates continuing to be between the 70s and the 90s, however, with less episodes of ectopy. Aspirin was given, and Lipitor was initially held for an initial transaminitis presumed to be secondary to shock liver. She had guaiac positive stools in the medical ICU, her hematocrit was stable around 33%, and her iron studies suggested anemia of chronic disease with possibly overlying iron deficiency. She had a normal random cortisol level of 35.3, and her Hemoglobin A1c was 6.5, so she was maintained thereafter only on insulin sliding scale and rarely required any coverage. The patient was kept on Lovenox and Protonix and her DISCHARGE MEDICATIONS include Aspirin 81 mg daily, iron sulfate 325 mg daily, gentamicin sulfate 50 mg IV q.8h. until 2/25 for a two-week course, penicillin G potassium 3 million units IV q.4h. until 0/12 for a six-week course, Lopressor 25 mg b.i.d., Caltrate plus D2 tablets p.o. daily, Lipitor 10 mg daily, and Protonix 40 mg daily. She was discharged to rehabilitation at Acanmingpeerra Virg Tantblu Medical Center in order to be able to get her antibiotic therapy, and her physicians will attempt to add the ACE back onto her medical regimen for better afterload reduction as her blood pressure tolerates, and potentially they will add her back on to the Lasix as well. She will require weekly lab draws to check her electrolytes and CBC while she is on the antibiotics.",Has the pt. ever been on caltrate plus d2 tablets before,"{'answer_end': [2086], 'answer_start': [2029], 'text': ['Caltrate plus D2 tablets p.o. daily, Lipitor 10 mg daily,']}"
1,"A 58 year old female smoker with a history of Coronary Artery Disease (CAD), Cirrhosis, Diabetes Mellitus Type II (DMII), Hypertension (HTN), and Hyperlipidemia was admitted to the CCU after an elective cardiac catheterization following an abnormal stress test. The cath showed impaired flow in the inferior and posterolateral zones due to obstructive degenerative disease in the SVGs to the RCA and LCF-OM, and a stent was placed in the RCA graft though there was extensive calcification and difficulty obtaining full stent expansion. After the stent deployment there was poor reflow accompanied by mild chest pain and EKG changes, without hemodynamic embarrassment. The patient experienced jaw and chest pain post-procedure which she described as different from previous episodes of angina. The pump-function was preserved, BP low-normal, and rhythm was NSR on telemetry. For pulmonary issues, the patient had a chronic cough due to post nasal drip which was taken off of her antihistamine on admission and CXR was normal with no acute changes. There were no renal issues during the hospital course and the patient was on Lantus, Novolog SS, and FS Glu monitored while in the hospital. Heme-wise, the patient had a cath and subsequent 

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [ ]:
max_length = 512 #1000 #max_length = 384 # The maximum length of a feature (question and context)
doc_stride = 172 #doc_stride = 128 # The authorized overlap between two part of the context when splitting it is needed.

In [ ]:
for i, example in enumerate(datasets["train"]):
    if len(tokenizer(example["question"], example["context"])["input_ids"]) > max_length: #384:
        break
example = datasets["train"][i]

Token indices sequence length is longer than the specified maximum sequence length for this model (1173 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
len(tokenizer(example["question"], example["context"])["input_ids"])

1173

In [ ]:
len(tokenizer(example["question"], example["context"], max_length=max_length, truncation="only_second")["input_ids"])

512

In [ ]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    stride=doc_stride
)

In [ ]:
[len(x) for x in tokenized_example["input_ids"]]

[512, 512, 512, 204]

In [ ]:
for x in tokenized_example["input_ids"][:2]:
    print(tokenizer.decode(x))

[CLS] What is her current dose of lasix ( furosemide ) [SEP] The patient was admitted on 5 / 5 / 2006 with a history of mechanical fall, with the attending physician being Dr. Clemente Armand Bolstad, with a full code status and disposition of Rehabilitation. Medications on Admission included Amiodarone 100 QD, Colace 100 bid, lasix 40mg QD, Glyburide 5mg bid, Plaquenil 200mg bid, Isordil 20mg tid, Lisinopril 20mg QD, Coumadin 5mg 3dys / week, 2. 5mg 4dys / week, Norvasc 10mg QD, Neurontin 300mg TID, with APAP prn. An override was added on 10 / 2 / 06 by Gerad E. Dancy, PA for POTENTIALLY SERIOUS INTERACTION : AMIODARONE HCL & WARFARIN with the reason for override being monitoring. The patient was rehydrated with IVF and PO's were encouraged, holding Glypizide while in house, Novolog sliding scale was started on 1 / 2, Low dose NPH 6 units BID was started on 1 / 2, bridged with lovenox and INR therapeutic 1 / 2 and restarted on home regimen of 5 / 2. 5mg variable dose. Pain was control

In [ ]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    stride=doc_stride
)
print(tokenized_example["offset_mapping"][0][:100])

[(0, 0), (0, 4), (5, 7), (8, 11), (12, 19), (20, 24), (25, 27), (28, 31), (31, 33), (34, 35), (36, 39), (39, 42), (42, 44), (44, 46), (47, 48), (0, 0), (0, 3), (4, 11), (12, 15), (16, 24), (25, 27), (28, 29), (29, 30), (30, 31), (31, 32), (32, 36), (37, 41), (42, 43), (44, 51), (52, 54), (55, 65), (66, 70), (70, 71), (72, 76), (77, 80), (81, 90), (91, 100), (101, 106), (107, 109), (109, 110), (111, 118), (118, 119), (120, 126), (127, 129), (129, 131), (131, 134), (134, 135), (136, 140), (141, 142), (143, 147), (148, 152), (153, 159), (160, 163), (164, 175), (176, 178), (179, 181), (181, 193), (193, 194), (195, 197), (197, 205), (205, 206), (207, 209), (210, 212), (212, 219), (220, 228), (229, 231), (231, 233), (233, 236), (236, 239), (240, 243), (244, 245), (245, 246), (246, 247), (248, 252), (252, 254), (255, 258), (259, 262), (262, 263), (264, 267), (267, 269), (270, 272), (272, 273), (273, 274), (275, 276), (276, 277), (277, 278), (279, 280), (280, 282), (282, 285), (285, 288), (289

In [ ]:
first_token_id = tokenized_example["input_ids"][0][1]
offsets = tokenized_example["offset_mapping"][0][1]
print(tokenizer.convert_ids_to_tokens([first_token_id])[0], example["question"][offsets[0]:offsets[1]])

What What


In [ ]:
sequence_ids = tokenized_example.sequence_ids()
print(sequence_ids)

[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [ ]:
example["answers"]["answer_start"][0]

2564

In [ ]:
answers = example["answers"]
start_char = answers["answer_start"][0]
end_char = start_char + len(answers["text"][0])

# Start token index of the current span in the text.
token_start_index = 0
while sequence_ids[token_start_index] != 1:
    token_start_index += 1

# End token index of the current span in the text.
token_end_index = len(tokenized_example["input_ids"][0]) - 1
while sequence_ids[token_end_index] != 1:
    token_end_index -= 1

# Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
offsets = tokenized_example["offset_mapping"][0]
if (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
    # Move the token_start_index and token_end_index to the two ends of the answer.
    # Note: we could go after the last offset if the answer is the last word (edge case).
    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
        token_start_index += 1
    start_position = token_start_index - 1
    while offsets[token_end_index][1] >= end_char:
        token_end_index -= 1
    end_position = token_end_index + 1
    print(start_position, end_position)
else:
    print("The answer is not in this feature.")

The answer is not in this feature.


In [ ]:
print(tokenizer.decode(tokenized_example["input_ids"][0][start_position: end_position+1]))
print(answers["text"][0])

NameError: ignored

In [ ]:
pad_on_right = tokenizer.padding_side == "right"

In [ ]:
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
    # left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [ ]:
features = prepare_train_features(datasets['train'][:5])

In [ ]:
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)

Map:   0%|          | 0/130956 [00:00<?, ? examples/s]

Map:   0%|          | 0/32739 [00:00<?, ? examples/s]

In [ ]:
#FINE TUNING THE MODEL
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at deepset/bert-base-cased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-squad",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

In [ ]:
from transformers import default_data_collator

data_collator = default_data_collator

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.175500,0.101393
2,0.059900,0.034424
3,0.027800,0.021719


TrainOutput(global_step=52062, training_loss=0.2206359952407722, metrics={'train_runtime': 26891.5469, 'train_samples_per_second': 30.976, 'train_steps_per_second': 1.936, 'total_flos': 2.176557563162235e+17, 'train_loss': 0.2206359952407722, 'epoch': 3.0})

In [ ]:
trainer.save_model('/content/drive/MyDrive/Colab Notebooks/dataset/medical_trained_model') #'/content/drive/MyDrive/Colab Notebooks/dataset/medical_trained_model'#tu_modelo.h5'

In [ ]:
from transformers import pipeline, AutoModelForQuestionAnswering, AutoTokenizer

# Suponiendo que estás utilizando un modelo BERT para Question Answering
model_path = '/content/drive/MyDrive/Colab Notebooks/dataset/medical_trained_model'

# Cargar el modelo desde el directorio
loaded_model = AutoModelForQuestionAnswering.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Crear un pipeline para utilizar el modelo cargado
qa_pipeline = pipeline("question-answering", model=loaded_model, tokenizer=tokenizer)

question = "Has the pt. ever been on caltrate plus d2 tablets before"
context = "The patient is a 76-year-old female with a history of mitral regurgitation, congestive heart failure, recurrent UTIs, and uterine prolapse who presented with chills and hypotension and was admitted to the Medical ICU for treatment of septic shock. Mean arterial pressures were kept above 65 with Levophed and antibiotics were changed to penicillin 3 million units IV q.4h. and gentamicin 50 mg IV q.8h. An ATEE on 10/19 showed severe mitral regurgitation with posterior leaflet calcifications and linear density concerning for endocarditis, for which a PICC line was placed on 1/19 for a six-week course of penicillin 3 million units IV q.4h. and two-week course of gentamicin 50 mg IV q.8h. until 2/25. The patient was initially treated with Levophed for her hypotension until 11/0, and was placed on Levofloxacin and Vancomycin to treat Gram-positive cocci bacteremia and UTI. She was maintained on telemetry and was found to be a normal sinus rhythm with ectopy, including short once of nonsustained ventricular tachycardia. She was started on Lopressor 12.5 mg t.i.d. on 3/18, and this was increased to 25 mg b.i.d. at discharge, with her heart rates continuing to be between the 70s and the 90s, however, with less episodes of ectopy. Aspirin was given, and Lipitor was initially held for an initial transaminitis presumed to be secondary to shock liver. She had guaiac positive stools in the medical ICU, her hematocrit was stable around 33%, and her iron studies suggested anemia of chronic disease with possibly overlying iron deficiency. She had a normal random cortisol level of 35.3, and her Hemoglobin A1c was 6.5, so she was maintained thereafter only on insulin sliding scale and rarely required any coverage. The patient was kept on Lovenox and Protonix and her DISCHARGE MEDICATIONS include Aspirin 81 mg daily, iron sulfate 325 mg daily, gentamicin sulfate 50 mg IV q.8h. until 2/25 for a two-week course, penicillin G potassium 3 million units IV q.4h. until 0/12 for a six-week course, Lopressor 25 mg b.i.d., Caltrate plus D2 tablets p.o. daily, Lipitor 10 mg daily, and Protonix 40 mg daily. She was discharged to rehabilitation at Acanmingpeerra Virg Tantblu Medical Center in order to be able to get her antibiotic therapy, and her physicians will attempt to add the ACE back onto her medical regimen for better afterload reduction as her blood pressure tolerates, and potentially they will add her back on to the Lasix as well. She will require weekly lab draws to check her electrolytes and CBC while she is on the antibiotics."
result = qa_pipeline(question=question, context=context)
print(result)

{'score': 0.9999037981033325, 'start': 2029, 'end': 2065, 'answer': 'Caltrate plus D2 tablets p.o. daily,'}
